# DEEP PEEVED: Neural Nets for Volcano Prediction


## Prepare dataset/dataloader

In [ ]:
from __future__ import absolute_import, print_function

import os
import os.path as osp
from glob import glob

import cv2
import numpy as np
import scipy.io as sio
import torch
from PIL import Image
from torch.utils import data
from collections import Counter


class Eruptions(data.Dataset):
    """PuuOo Eruption Dataset"""

    def __init__(self, root, split):
        self.root  = root
        self.split = split
        
    def _(self):
        # Create data list via {train, test, all}.txt
        if self.split in ["train", "val", "test"]:
            file_path = osp.join(self.root, self.split + ".txt")
            with open(file_path, 'r') as f:
                files = f.readlines()
                self.files = [id.strip() for id in files]
        else:
            raise ValueError("Invalid split name: {}".format(self.split))

    def _get_label_weights(self):
        # Get weights for a given dataset
        ids = [self._get_disaster_id(sample_name) for sample_name in self.files]
        id_counts = Counter(ids)

        total = len(ids)

        weights = [min(total/id_counts[i], 15) for i in range(self.num_classes)]

        return weights

    def _load_data(self, index):
        # Set paths
        return sample_id, pre_image, post_image
    
    def __getitem__(self, index):
        image_id, pre_image, post_image = self._load_data(index)

        return image_id, combined_img.astype(np.float32)

    def __len__(self):
        return len(self.files)


if __name__ == "__main__":
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import torchvision
    import yaml
    from torchvision.utils import make_grid
    from tqdm import tqdm

    kwargs = {"nrow": 10, "padding": 50}
    batch_size = 1

    dataset = Disaster_ds(
        root="../../../../data/train",
        split="train",
    )
    print(dataset)

    loader = data.DataLoader(dataset, batch_size=batch_size)

    for i, xin tqdm(enumerate(loader), total=np.ceil(len(dataset) / batch_size), leave=False
    ):
        print(i)
        if i == 0:
            print(image_ids)
    

